## Purpouse of this notebook is to create new datasets from the one we have,
### Firstly create separate folders for various depths,  one for averaged, and one for our base dataset
### Then perform cleaning based on data_cleaning.ipynb

In [42]:
from os import listdir, makedirs
from os.path import isfile, join, exists


import pandas as pd
import numpy as np

from functools import reduce

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score


In [43]:
DATA_PATH = '2022.11.07-merged-single-observation'
FILENAMES = [f for f in listdir(DATA_PATH) if (isfile(join(DATA_PATH, f)) and not f.startswith('.') and f.endswith('.csv'))]
DEPTHS = [ 100, 500, 1000, 1500, 2000, 2500]
MERGED_NAME = 'CLEANED_MERGED_DATA'

In [44]:
### function we will split by 

## helpful filters and statistical functions

def fix_date_formatting(df):
    df['DATE'] = pd.to_datetime(df.DATE, infer_datetime_format=True)
    return df

def filter_by_months(df, months):
    return df[df['DATE'].dt.month == any(months)]

def group_by_year(df, par):
    df =  df.groupby(df['DATE'].dt.year)[par].mean()
    return pd.DataFrame({"DATE":df.index, par:df.values})

def group_by_date(df, par):
    df =  df.groupby(df['DATE'].dt.date)[par].mean()
    return pd.DataFrame({"DATE":df.index, par:df.values})

def group_by_month(df, par):
    #df = df.drop('DEPTH', axis=1)
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.groupby(pd.Grouper(freq='M', key='DATE')).mean()
    df = df.reset_index()
    return df

def filter_by_depth(df, depth):
    df = df.loc[df['DEPTH'] == depth]
    return df

def filter_by_depth_range(df, low, high):
    df = df.loc[low <= df['DEPTH'] <= high]
    return df

def group_by_depth(df, par):
    df =  df.groupby(df['DEPTH'])[par].mean()
    return pd.DataFrame({"DEPTH":df.index, par:df.values})

def drop_outliers(df,param, quantile):
    q = df[param].quantile(quantile)
    return df[df[param] < q]

#return new dataframe with par replaced by it's movign average
def moving_averages(df, param, window_size):
    _df = df.copy()
    _df[param] = df[param].rolling(window=window_size).mean()
    return _df

def apply_features_transform(df, param, quantile=.95, depth=[100, 2500], moving_avg_window=6):
    df = fix_date_formatting(df)
    #df = filter_by_depth_range(df, depth[0], depth[1])
    df = df.drop('DEPTH', axis=1)
    df = drop_outliers(df, param, quantile)
    df = group_by_month(df, param)
    df = moving_averages(df, param, moving_avg_window)
    #df = group_by_year(df, param)
    
    return df

In [45]:
FILENAMES

['NO2.csv',
 'NH4.csv',
 'PH.csv',
 'SECCHI.csv',
 'CTDTMP.csv',
 'CHLORA.csv',
 'CTDSAL.csv',
 'PO4.csv',
 'OXY.csv',
 'TP.csv',
 'TN.csv',
 'SIO2.csv',
 'NO3.csv']

In [46]:
# create folders for data if they don't exist inside DATA_PATH
for d in ([str(d) for d in DEPTHS] + ['average', 'standard', 'deep']):
    if not exists(DATA_PATH +'/'+ d):
        makedirs(DATA_PATH + '/' + d)

for file in FILENAMES:
    ##read as csv
    dataframe = pd.read_csv(DATA_PATH + '/' + file,index_col=False, )
    ## get parameter from column name
    param = dataframe.columns[2]
    ## standard
    dataframe.to_csv(DATA_PATH + '/standard/' + file,index=False)
    dataframe['DATE'] = pd.to_datetime(dataframe['DATE'],dayfirst = True)
    for d in DEPTHS:
        depth_filtered = dataframe.loc[dataframe['DEPTH'] == d]
        #depth_filtered = group_by_month(depth_filtered, param)
        ## save to folder for this depth
        depth_filtered.to_csv(DATA_PATH+'/'+str(d)+'/'+file,index=False)
    ## do the same for averaged by depth
    average = apply_features_transform(dataframe, param)
    average.to_csv(DATA_PATH + '/average/' + file,index=False)
    ## and for deep only
    deep = dataframe.loc[2000 <= dataframe['DEPTH']]
    deep.to_csv(DATA_PATH+'/deep/'+file, index=False)

In [47]:
## create dataframe merging csv files in directory
def merge(path):
    dfs = {filename: pd.read_csv(path + '/' + filename,index_col=False )
           for filename in FILENAMES
           if filename.endswith('.csv') and not filename.startswith('.')}

    df_list=[]
    for df in dfs.values():
        df['DATE'] = pd.to_datetime(df['DATE'],dayfirst = True)
        df_list.append(df)
    
    on = ['DATE', 'DEPTH']
    if 'DEPTH' not in df_list[0].columns:
        on = ['DATE'] 
    df = reduce(lambda left,right: pd.merge(left.drop_duplicates(subset=on)              ,right.drop_duplicates(subset=on)              ,on=on              ,how='outer'), df_list)
    #df = reduce(lambda left,right: pd.merge(left.drop_duplicates(subset=['DATE','DEPTH']),right.drop_duplicates(subset=['DATE','DEPTH']),on=['DATE','DEPTH'],how='outer'), df_list)
    return df


In [48]:
merge(DATA_PATH + '/standard')

,DATE,DEPTH,NO2,NH4,PH,SECCHI,CTDTMP,CHLORA,CTDSAL,PO4,OXY,TP,TN,SIO2,NO3
0,1986-01-07,1000,1.05,5.6,NaN,NaN,2.47,NaN,17.89,1.37,356.8,NaN,NaN,23.1,8.4
1,1986-01-07,100,0.96,5.5,NaN,NaN,2.56,NaN,17.88,1.33,355.9,NaN,NaN,23.9,7.0
2,1986-01-07,2500,1.14,6.0,NaN,NaN,2.89,NaN,18.04,1.43,346.6,NaN,NaN,25.1,9.7
3,1986-01-07,500,1.02,5.5,NaN,NaN,2.56,NaN,17.89,1.32,356.8,NaN,NaN,23.7,8.6
4,1986-01-07,2000,1.03,5.8,NaN,NaN,2.63,NaN,17.95,1.36,338.1,NaN,NaN,24.5,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4763,1975-12-02,1000,NaN,NaN,NaN,NaN,6.70,8.5,19.20,NaN,296.9,1.58,NaN,NaN,NaN
4764,1975-12-02,2600,NaN,NaN,NaN,NaN,8.50,20.0,20.10,NaN,178.1,3.62,NaN,NaN,NaN
4765,1975-12-02,100,NaN,NaN,NaN,NaN,6.70,7.9,19.20,NaN,318.8,1.71,NaN,NaN,NaN
4766,1975-12-02,2000,NaN,NaN,NaN,NaN,6.70,8.6,19.20,NaN,306.3,1.52,NaN,NaN,NaN


In [49]:
## perform cleaning (see data_cleaning.ipynb)
for path in ([str(i) for i in DEPTHS] + ['average', 'standard','deep']):
    
    df = merge(DATA_PATH + '/' + path)
    
    # rest of loop is mostly copy of regression method from data_cleaning.ipynb
    # check missing values
    df_missing = df.drop(['DATE'],axis=1)
    missing = df_missing.isna().sum()
    missing = pd.DataFrame(data={'elements': missing.index,'missing':missing.values})
    missing = missing[~missing['missing'].isin([0])]
    missing['percentage'] =  missing['missing']/df_missing.shape[0]
    missing.sort_values(by='percentage',ascending=False)

    # check df data type
    #df_missing[missing['elements']].info()
    
    # check df data type
    df_missing[missing['elements']].info()
    if path != 'average':
        df_missing = df_missing.drop(['DEPTH'],axis=1)
    df_missing = df_missing.drop(['PH'],axis=1) # PH has too many missing values
    # df_missing.head
    
    X_missing = df_missing.copy()
    y_missing = df_missing.copy()
    y_missing.dropna(inplace=True) 
    X_missing = pd.DataFrame(X_missing)
    y_missing = pd.DataFrame(y_missing)

    # regression method
    X_missing_reg = X_missing.copy()
    sortindex = np.argsort(X_missing_reg.isnull().sum(axis=0)).values #sort missing columns

    for i in sortindex:
        df_ = X_missing_reg  
        fillc = df_.iloc[:, i]  
        df_ = pd.concat([df_.drop(df_.columns[i], axis=1), pd.DataFrame(y_missing)], axis=1)
        df_0 = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0).fit_transform(df_)
        #train and test dataset
        Ytrain = fillc[fillc.notnull()]  # not missing part is Y_train
        Ytest = fillc[fillc.isnull()] 
        Xtrain = df_0[Ytrain.index, :]
        Xtest = df_0[Ytest.index, :] 
        rfc = RandomForestRegressor(n_estimators=100) 
        rfc = rfc.fit(Xtrain, Ytrain)  
        Ypredict = rfc.predict(Xtest)
        # put prediction values back to df
        X_missing_reg.loc[X_missing_reg.iloc[:, i].isnull(), X_missing_reg.columns[i]] = Ypredict


    X_missing_reg.insert(0,column = 'DATE',value=df['DATE'])

    ##save csv
    X_missing_reg.to_csv(DATA_PATH  + '/' + path + '/' + MERGED_NAME + '.csv' ,index=False)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 739 entries, 0 to 738
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NO2     440 non-null    float64
 1   NH4     453 non-null    float64
 2   PH      125 non-null    float64
 3   SECCHI  454 non-null    float64
 4   CTDTMP  719 non-null    float64
 5   CHLORA  485 non-null    float64
 6   CTDSAL  715 non-null    float64
 7   PO4     588 non-null    float64
 8   OXY     714 non-null    float64
 9   TP      642 non-null    float64
 10  TN      425 non-null    float64
 11  SIO2    465 non-null    float64
 12  NO3     471 non-null    float64
dtypes: float64(13)
memory usage: 80.8 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 732 entries, 0 to 731
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NO2     438 non-null    float64
 1   NH4     447 non-null    float64
 2   PH      126 non-null    float64
 3   SEC